# Loading data

In [1]:
using JuMP
using CSV
using DataFrames
using HiGHS
using PlotlyJS
using StatsBase

In [2]:
lines = CSV.read("/Users/alexanderklaus/Desktop/Masterthesis/Code/data/lines.csv", DataFrame)
demand = CSV.read("/Users/alexanderklaus/Desktop/Masterthesis/Code/data/demand.csv", DataFrame)
#busses = CSV.read("/Users/alexanderklaus/Desktop/Masterthesis/Code/data/busses.csv", DataFrame)
bus_line_stops = CSV.read("/Users/alexanderklaus/Desktop/Masterthesis/Code/data/bus-lines.csv", DataFrame)

Row,bus_line_id,stop_ids,stop_x,stop_y
,Int64,Int64,Float64,Float64
1,1,1,2.2,36.8
2,1,2,4.4,40.1
3,1,3,12.5,42.7
4,1,4,20.2,44.0
5,2,1,40.9,48.2
6,2,2,46.2,40.7
7,2,3,56.6,38.2
8,3,1,44.6,15.2
9,3,2,52.0,20.4


In [3]:
lines

Row,line_id,bus_line_id,start_time
,Int64,Int64,Float64
1,1,1,20.0
2,2,1,40.0
3,3,1,60.0
4,1,2,20.0
5,2,2,35.0
6,3,2,50.0
7,1,3,16.0
8,2,3,32.0
9,3,3,48.0


In [4]:
demand

Row,demand_id,line_id,bus_line_id,origin_stop_id,destination_stop_id,demand
,Int64,Int64,Int64,Int64,Int64,Int64
1,1,1,1,1,4,1
2,2,1,1,2,3,2
3,3,1,1,2,4,1
4,4,2,1,3,4,1
5,5,2,1,1,3,2
6,6,1,2,1,2,4
7,7,3,2,1,3,1
8,8,1,3,1,6,2
9,9,1,3,2,5,1


# Data understanding

## Looking at the stops

In [5]:
using PlotlyJS
using DataFrames
using Statistics

# Gruppiere nach bus_line_id
grouped_data = groupby(bus_line_stops, :bus_line_id)

# Sammle alle Traces
traces = PlotlyJS.AbstractTrace[]  # use correct type

# Linien- und Beschriftungstraces erzeugen
for grp in grouped_data
    sort!(grp, :stop_ids)

    # Linien-Trace mit Markern
    push!(traces, scatter(
        x = grp.stop_x,
        y = grp.stop_y,
        mode = "lines+markers+text",
        name = "Linie $(unique(grp.bus_line_id)[1])",
        marker = attr(size = 8),
        text = string.(grp.stop_ids),
        textposition = "top center",
        textfont = attr(color="black", size=10)
    ))

    # Distanzen zwischen benachbarten Haltestellen
    for i in 1:(nrow(grp) - 1)
        x1, y1 = grp.stop_x[i], grp.stop_y[i]
        x2, y2 = grp.stop_x[i+1], grp.stop_y[i+1]
        dist = round(sqrt((x2 - x1)^2 + (y2 - y1)^2), digits=2)
        mx, my = mean([x1, x2]), mean([y1, y2])

        # Distanztext hinzufügen
        push!(traces, scatter(
            x = [mx], y = [my],
            mode = "text",
            text = ["d = $dist"],
            textposition = "bottom center",
            textfont = attr(size = 9, color = "gray"),
            showlegend = false
        ))
    end
end

# Layout festlegen
layout = Layout(
    title = "Buslinien und ihre Haltestellen",
    xaxis_title = "X-Koordinate",
    yaxis_title = "Y-Koordinate",
    legend = attr(x=1.02, y=1)
)

# Plot erstellen
p = Plot(traces, layout)

# In HTML-Datei speichern (im gleichen Ordner wie Code)
PlotlyJS.savefig(p, "/Users/alexanderklaus/Desktop/Masterthesis/Code/output/buslinien_plot.html")


"/Users/alexanderklaus/Desktop/Masterthesis/Code/output/buslinien_plot.html"

# Preprocessing

## Input settings

### Adding the depot

In [6]:
depot=(bus_line_id=0,stop_ids=0,stop_x=30,stop_y=30)
insert!(bus_line_stops,1,depot)
depot=(line_id=0,bus_line_id=0, start_time=0)
insert!(lines,1,depot)

Row,line_id,bus_line_id,start_time
,Int64,Int64,Float64
1,0,0,0.0
2,1,1,20.0
3,2,1,40.0
4,3,1,60.0
5,1,2,20.0
6,2,2,35.0
7,3,2,50.0
8,1,3,16.0
9,2,3,32.0


### Bus velocity

In [7]:
bus_velocity = 100 #in km/h

100

## Processing data

### Creating nodes for all stops, on every busline, for every tour

In [8]:
nodes = []

for r in eachrow(lines)
    for row in eachrow(bus_line_stops)
        if r.bus_line_id == row.bus_line_id
            start_time_temp=0.0
            if row.stop_ids == 1
                start_time_temp = r.start_time
            else
                # alle Stopps dieser Linie sortieren
                stops = sort(filter(r2 -> r2.bus_line_id == row.bus_line_id, eachrow(bus_line_stops)), by = r2 -> r2.stop_ids)
                
                # Travel time vom ersten Knoten bis aktuellen Knoten:
                total_distance = 0.0
                for k in 1:(row.stop_ids - 1)
                    x1, y1 = stops[k].stop_x, stops[k].stop_y
                    x2, y2 = stops[k+1].stop_x, stops[k+1].stop_y
                    total_distance += sqrt((x2 - x1)^2 + (y2 - y1)^2)
                end
                travel_time = (total_distance / bus_velocity) * 60  # Minuten

                # Startzeit = Startzeit erster Knoten + Travel Time
                start_time_temp = r.start_time + travel_time
            end
            push!(nodes, (
                bus_line_id = row.bus_line_id,
                line_id     = r.line_id,
                stop_id     = row.stop_ids,
                coord_x     = row.stop_x,
                coord_y     = row.stop_y,
                start_time  = start_time_temp
            ))
                
        end
    end
end

In [9]:
print("Nodes:")
nodes

Nodes:

68-element Vector{Any}:
 (bus_line_id = 0, line_id = 0, stop_id = 0, coord_x = 30.0, coord_y = 30.0, start_time = 0.0)
 (bus_line_id = 1, line_id = 1, stop_id = 1, coord_x = 2.2, coord_y = 36.8, start_time = 20.0)
 (bus_line_id = 1, line_id = 1, stop_id = 2, coord_x = 4.4, coord_y = 40.1, start_time = 22.379663841806234)
 (bus_line_id = 1, line_id = 1, stop_id = 3, coord_x = 12.5, coord_y = 42.7, start_time = 27.483897378783162)
 (bus_line_id = 1, line_id = 1, stop_id = 4, coord_x = 20.2, coord_y = 44.0, start_time = 32.16927890006828)
 (bus_line_id = 1, line_id = 2, stop_id = 1, coord_x = 2.2, coord_y = 36.8, start_time = 40.0)
 (bus_line_id = 1, line_id = 2, stop_id = 2, coord_x = 4.4, coord_y = 40.1, start_time = 42.379663841806234)
 (bus_line_id = 1, line_id = 2, stop_id = 3, coord_x = 12.5, coord_y = 42.7, start_time = 47.48389737878316)
 (bus_line_id = 1, line_id = 2, stop_id = 4, coord_x = 20.2, coord_y = 44.0, start_time = 52.16927890006828)
 (bus_line_id = 1, line_id = 3, stop

### Creating connections

In [10]:
function eucleadian_distance(x1, y1, x2, y2)
    return sqrt((x1 - x2)^2 + (y1 - y2)^2)
end

eucleadian_distance (generic function with 1 method)

In [11]:
# Define a struct to hold the travel_time and end_time
struct Connection
    distance::Float64
    travel_time::Float64
    end_time::Float64
end

# Update the dictionary to use the struct as the value type
connections_dict = Dict()

for node1 in nodes
    for node2 in nodes
        if node1 != node2
            #skip if the nodes are the same
            origin_node = (node1.bus_line_id, node1.line_id, node1.stop_id)
            goal_node = (node2.bus_line_id, node2.line_id, node2.stop_id)
            
            if node1.bus_line_id == node2.bus_line_id
                if node1.line_id == node2.line_id
                    if node1.stop_id == node2.stop_id
                        # Nodes are the same
                        break
                    else
                        # Nodes are on the same tour
                        stops = sort(filter(n -> n.bus_line_id == node1.bus_line_id, nodes), by = n -> n.stop_id)
                        start_index = findfirst(n -> n.stop_id == node1.stop_id, stops)
                        end_index = findfirst(n -> n.stop_id == node2.stop_id, stops)
                        
                        if start_index < end_index
                            #if origin node comes before goal node
                            total_distance = 0.0
                            for i in start_index:(end_index - 1)
                                x1, y1 = stops[i].coord_x, stops[i].coord_y
                                x2, y2 = stops[i + 1].coord_x, stops[i + 1].coord_y
                                total_distance += eucleadian_distance(x1, y1, x2, y2)
                            end
                            distance= total_distance
                            travel_time = (total_distance / bus_velocity) * 60 #travel_time in minutes
                            end_time = node1.start_time + travel_time
                            connections_dict[(origin_node, goal_node)] = Connection(distance, travel_time, end_time)
                        end
                    end
                else
                    if node1.stop_id != node2.stop_id
                        #nodes are on the same bus line but not the same tour
                        distance = eucleadian_distance(node1.coord_x, node1.coord_y, node2.coord_x, node2.coord_y)
                        travel_time = (distance / bus_velocity) * 60 #travel_time in minutes
                        end_time = node1.start_time + travel_time
                        connections_dict[(origin_node, goal_node)] = Connection(distance, travel_time, end_time)
                    end
                end
            else
                # Nodes are on different bus lines
                distance = eucleadian_distance(node1.coord_x, node1.coord_y, node2.coord_x, node2.coord_y)
                travel_time = (distance / bus_velocity) * 60 #travel_time in minutes
                end_time = node1.start_time + travel_time
                connections_dict[(origin_node, goal_node)] = Connection(distance, travel_time, end_time)
            end
        end
    end
end

println("Connections dictionary:")
sort(collect(connections_dict), by = x -> x.first)

Connections dictionary:


4278-element Vector{Pair{Any, Any}}:
 ((0, 0, 0), (1, 1, 1)) => Connection(28.619573721493477, 17.17174423289609, 17.17174423289609)
 ((0, 0, 0), (1, 1, 2)) => Connection(27.520356102347225, 16.512213661408335, 16.512213661408335)
 ((0, 0, 0), (1, 1, 3)) => Connection(21.622673285234647, 12.973603971140788, 12.973603971140788)
 ((0, 0, 0), (1, 1, 4)) => Connection(17.089177862027185, 10.253506717216311, 10.253506717216311)
 ((0, 0, 0), (1, 2, 1)) => Connection(28.619573721493477, 17.17174423289609, 17.17174423289609)
 ((0, 0, 0), (1, 2, 2)) => Connection(27.520356102347225, 16.512213661408335, 16.512213661408335)
 ((0, 0, 0), (1, 2, 3)) => Connection(21.622673285234647, 12.973603971140788, 12.973603971140788)
 ((0, 0, 0), (1, 2, 4)) => Connection(17.089177862027185, 10.253506717216311, 10.253506717216311)
 ((0, 0, 0), (1, 3, 1)) => Connection(28.619573721493477, 17.17174423289609, 17.17174423289609)
 ((0, 0, 0), (1, 3, 2)) => Connection(27.520356102347225, 16.512213661408335, 16.512213

### Dictionary mit Stopp-IDs je Linie

In [12]:
line_stops_dict = Dict{Int, Vector{Int}}()

for row in eachrow(bus_line_stops)
    l = row.bus_line_id
    stop = row.stop_ids
    # Depot-Zeile überspringen
    if l == 0 && stop == 0
        continue
    end
    if haskey(line_stops_dict, l)
        push!(line_stops_dict[l], stop)
    else
        line_stops_dict[l] = [stop]
    end
end
println("Dictionary of lines with their respective stop-sequences:")
sort(line_stops_dict)

Dictionary of lines with their respective stop-sequences:


OrderedCollections.OrderedDict{Int64, Vector{Int64}} with 5 entries:
  1 => [1, 2, 3, 4]
  2 => [1, 2, 3]
  3 => [1, 2, 3, 4, 5, 6]
  4 => [1, 2]
  5 => [1, 2, 3, 4, 5]

### Identifying Critical pairs

In [60]:
# Initialisierung
critical_pairs_dict = Dict{Tuple{Tuple{Int, Int, Int}, Tuple{Int, Int, Int}}, Tuple{Tuple{Int, Int, Int}, Tuple{Int, Int, Int}}}()
A_set = Set{Tuple{Tuple{Int, Int, Int}, Tuple{Int, Int, Int}}}()
V_all = []
V_flow = Set{Tuple{Int, Int, Int}}()
dup_start_set=[]
dup_end_set=[]
duplikate_counter = 1

# Iteriere über alle Buslinien
for bus_line in unique(demand.bus_line_id)
    line_trips = filter(row -> row.bus_line_id == bus_line, demand)
    #jetz gehe jede alle touren dieser bus linie durch
    for i in 1:size(line_trips, 1)-1
        for j in i+1:size(line_trips, 1)
            trip1 = line_trips[i, :]
            trip2 = line_trips[j, :]

            if trip1.line_id == trip2.line_id

                o1, d1 = trip1.origin_stop_id, trip1.destination_stop_id
                o2, d2 = trip2.origin_stop_id, trip2.destination_stop_id

                # check if trip1 enthält trip2
                if o1 < o2 && d1 > d2

                    inner_trip = ((trip2.bus_line_id, trip2.line_id, o2), (trip2.bus_line_id, trip2.line_id, d2))
                    outer_trip = ((trip1.bus_line_id, trip1.line_id, o1), (trip1.bus_line_id, trip1.line_id, d1))
                    #println("Critical pair found: ", inner_trip, " contained in ", outer_trip)
                    critical_pairs_dict[inner_trip] = outer_trip

                    # Duplikate erzeugen
                    dup1_start = inner_trip[1]
                    dup1_end = (trip2.bus_line_id, trip2.line_id,d1) #setze das ende vom ersten duplikat auf das ende des äußeren trips
                    dup2_start = inner_trip[1] #ist gleich dup1_start
                    dup2_end = inner_trip[2]
                    dup1 = (dup1_start, dup1_end)
                    dup2 = (dup2_start, dup2_end)
                    source_node = (0,duplikate_counter,0)
                    sink_node   = (0,0,duplikate_counter)
                    duplikate_counter += 1

                    push!(V_all, source_node)
                    push!(V_all, sink_node)
                    push!(A_set, dup1)
                    push!(A_set, dup2)
                    push!(A_set, (source_node, dup1_start))
                    push!(dup_start_set,(source_node, dup1_start))
                    push!(A_set, (dup1_end, sink_node))
                    push!(A_set, (dup2_end, sink_node))
                    push!(dup_end_set,(dup1_end, sink_node)) #für constraint bildung
                    push!(dup_end_set,(dup2_end, sink_node)) #für constraint bildung


                # check if trip2 enthält trip1
                elseif o2 < o1 && d2 > d1
                    inner_trip = ((trip1.bus_line_id, trip1.line_id, o1), (trip1.bus_line_id, trip1.line_id, d1))
                    outer_trip = ((trip2.bus_line_id, trip2.line_id, o2), (trip2.bus_line_id, trip2.line_id, d2))
                    #println("Critical pair found: ", inner_trip, " contained in ", outer_trip)
                    critical_pairs_dict[inner_trip] = outer_trip

                    #Duplikate erzeugen
                    dup1_start = inner_trip[1]
                    dup1_end = (trip1.bus_line_id, trip1.line_id,d2) #setze das ende vom ersten duplikat auf das ende des äußeren trips
                    dup2_start = inner_trip[1] #ist gleich dup1_start
                    dup2_end = inner_trip[2]
                    dup1 = (dup1_start, dup1_end)
                    dup2 = (dup2_start, dup2_end)
                    source_node = (0,duplikate_counter,0)
                    sink_node   = (0,0,duplikate_counter)
                    duplikate_counter += 1

                    push!(V_all, source_node)
                    push!(V_all, sink_node)
                    push!(A_set, dup1)
                    push!(A_set, dup2)
                    push!(A_set, (source_node, dup1_start))
                    push!(dup_start_set,(source_node, dup1_start))
                    push!(A_set, (dup1_end, sink_node))
                    push!(A_set, (dup2_end, sink_node))
                    push!(dup_end_set,(dup1_end, sink_node)) #für constraint bildung
                    push!(dup_end_set,(dup2_end, sink_node)) #für constraint bildung
                    
                #check if trip1 und trip2 sich überschneiden, aber nicht enthalten
                elseif (o1 < o2 && d1 < d2 && d1 > o2)
                    # Verknüpfe: erst origin1 → origin2, dann destination1 → destination2
                    push!(A_set, ((trip1.bus_line_id, trip1.line_id, o1), (trip2.bus_line_id, trip2.line_id, o2)))
                    push!(A_set, ((trip1.bus_line_id, trip1.line_id, d1), (trip2.bus_line_id, trip2.line_id, d2)))
                elseif (o2 < o1 && d2 < d1 && d2 > o1)
                    # Fall umgekehrt: trip2 beginnt früher, endet aber innerhalb von trip1
                    push!(A_set, ((trip2.bus_line_id, trip2.line_id, o2), (trip1.bus_line_id, trip1.line_id, o1)))
                    push!(A_set, ((trip2.bus_line_id, trip2.line_id, d2), (trip1.bus_line_id, trip1.line_id, d1)))
                elseif d1==d2
                    if o1<o2
                        push!(A_set, ((trip1.bus_line_id, trip1.line_id, o1), (trip2.bus_line_id, trip2.line_id, o2)))
                    elseif o2<o1
                        push!(A_set, ((trip2.bus_line_id, trip2.line_id, o2), (trip1.bus_line_id, trip1.line_id, o1)))
                    end
                elseif o1==o2 
                    if d1<d2
                        push!(A_set, ((trip1.bus_line_id, trip1.line_id, d1), (trip2.bus_line_id, trip2.line_id, d2)))
                    elseif d2<d1
                        push!(A_set, ((trip2.bus_line_id, trip2.line_id, d2), (trip1.bus_line_id, trip1.line_id, d1)))
                    end
                end
            else
                continue
            end
        end
    end
end

# Ausgabe prüfen
println("Critical pairs dictionary:")
for (k, v) in critical_pairs_dict
    println("Inner trip: ", k, "  contained in outer trip: ", v)
end
sorted_A=sort(collect(A_set), by = x -> x[1])
println("\n\n A_set:")
for a in sorted_A
    println(a)
end


Critical pairs dictionary:
Inner trip: ((5, 3, 2), (5, 3, 3))  contained in outer trip: ((5, 3, 1), (5, 3, 4))
Inner trip: ((1, 1, 2), (1, 1, 3))  contained in outer trip: ((1, 1, 1), (1, 1, 4))
Inner trip: ((3, 1, 2), (3, 1, 5))  contained in outer trip: ((3, 1, 1), (3, 1, 6))


 A_set:
((0, 1, 0), (1, 1, 2))
((0, 2, 0), (3, 1, 2))
((0, 3, 0), (5, 3, 2))
((1, 1, 1), (1, 1, 2))
((1, 1, 2), (1, 1, 3))
((1, 1, 2), (1, 1, 4))
((1, 1, 3), (1, 1, 4))
((1, 1, 3), (0, 0, 1))
((1, 1, 4), (0, 0, 1))
((3, 1, 2), (3, 1, 6))
((3, 1, 2), (3, 1, 5))
((3, 1, 5), (0, 0, 2))
((3, 1, 6), (0, 0, 2))
((5, 3, 1), (5, 3, 2))
((5, 3, 2), (5, 3, 3))
((5, 3, 2), (5, 3, 4))
((5, 3, 3), (5, 3, 4))
((5, 3, 3), (0, 0, 3))
((5, 3, 4), (0, 0, 3))


### Populating set A

In [61]:
depot = (0, 0, 0)
for row in eachrow(demand)
    from_node = (row.bus_line_id, row.line_id, row.origin_stop_id)
    to_node   = (row.bus_line_id, row.line_id, row.destination_stop_id)
    trip_key  = (from_node, to_node)
    # Depot -> origin
    push!(A_set, (depot, from_node))
    # destination -> Depot
    push!(A_set, (to_node, depot))
    if !((from_node, to_node) in A_set)
        # origin -> destination
        if haskey(connections_dict, (from_node, to_node))
            push!(A_set, (from_node, to_node))
        else
            println("WARNUNG - Verbindung fehlt in connections_dict: ", (from_node, to_node))
        end
    else
        connection=(from_node, to_node)
        println(" Verbindung $connection bereits in A_set.")
    end
end


 Verbindung ((1, 1, 2), (1, 1, 3)) bereits in A_set.
 Verbindung ((1, 1, 2), (1, 1, 4)) bereits in A_set.
 Verbindung ((3, 1, 2), (3, 1, 5)) bereits in A_set.
 Verbindung ((5, 3, 2), (5, 3, 4)) bereits in A_set.
 Verbindung ((5, 3, 2), (5, 3, 3)) bereits in A_set.


In [62]:
sorted_A=sort(collect(A_set))
println("\n\n A_set:")
for a in sorted_A
    println(a)
end



 A_set:
((0, 0, 0), (1, 1, 1))
((0, 0, 0), (1, 1, 2))
((0, 0, 0), (1, 2, 1))
((0, 0, 0), (1, 2, 3))
((0, 0, 0), (2, 1, 1))
((0, 0, 0), (2, 3, 1))
((0, 0, 0), (3, 1, 1))
((0, 0, 0), (3, 1, 2))
((0, 0, 0), (3, 2, 1))
((0, 0, 0), (3, 2, 4))
((0, 0, 0), (4, 2, 1))
((0, 0, 0), (4, 4, 1))
((0, 0, 0), (5, 1, 3))
((0, 0, 0), (5, 3, 1))
((0, 0, 0), (5, 3, 2))
((0, 1, 0), (1, 1, 2))
((0, 2, 0), (3, 1, 2))
((0, 3, 0), (5, 3, 2))
((1, 1, 1), (1, 1, 2))
((1, 1, 1), (1, 1, 4))
((1, 1, 2), (1, 1, 3))
((1, 1, 2), (1, 1, 4))
((1, 1, 3), (0, 0, 0))
((1, 1, 3), (0, 0, 1))
((1, 1, 3), (1, 1, 4))
((1, 1, 4), (0, 0, 0))
((1, 1, 4), (0, 0, 1))
((1, 2, 1), (1, 2, 3))
((1, 2, 3), (0, 0, 0))
((1, 2, 3), (1, 2, 4))
((1, 2, 4), (0, 0, 0))
((2, 1, 1), (2, 1, 2))
((2, 1, 2), (0, 0, 0))
((2, 3, 1), (2, 3, 3))
((2, 3, 3), (0, 0, 0))
((3, 1, 1), (3, 1, 6))
((3, 1, 2), (3, 1, 5))
((3, 1, 2), (3, 1, 6))
((3, 1, 5), (0, 0, 0))
((3, 1, 5), (0, 0, 2))
((3, 1, 6), (0, 0, 0))
((3, 1, 6), (0, 0, 2))
((3, 2, 1), (3, 2, 2))
(

In [68]:

# Iteriere über alle Buslinien
for bus_line in unique(demand.bus_line_id)
    line_trips = filter(row -> row.bus_line_id == bus_line, demand)
    println(line_trips)
    for line_id in unique(line_trips.line_id)
        single_line_trips = filter(row -> row.line_id == line_id, line_trips)
        println(single_line_trips)
        first_stop_of_tour=minimum(unique(single_line_trips.origin_stop_id))
        last_stop_of_tour=maximum(unique(single_line_trips.destination_stop_id))   
        
    end


end

5×6 DataFrame
 Row │ demand_id  line_id  bus_line_id  origin_stop_id  destination_stop_id  demand 
     │ Int64      Int64    Int64        Int64           Int64                Int64  
─────┼──────────────────────────────────────────────────────────────────────────────
   1 │         1        1            1               1                    4       1
   2 │         2        1            1               2                    3       2
   3 │         3        1            1               2                    4       1
   4 │         4        2            1               3                    4       1
   5 │         5        2            1               1                    3       2
3×6 DataFrame
 Row │ demand_id  line_id  bus_line_id  origin_stop_id  destination_stop_id  demand 
     │ Int64      Int64    Int64        Int64           Int64                Int64  
─────┼──────────────────────────────────────────────────────────────────────────────
   1 │         1        1            1    

### Populating set V

In [17]:
using OrderedCollections  
push!(V_set, (0, 0, 0))  # Add the depot D
for row in eachrow(demand)
    origin_tuple = (row.bus_line_id, row.line_id, row.origin_stop_id)
    destination_tuple = (row.bus_line_id, row.line_id, row.destination_stop_id)
    push!(V_set, origin_tuple)
    push!(V_set, destination_tuple)
end

V_set = sort(collect(V_set))
full_output = join(string.(V_set), "\n")
println("V_set:")
println(full_output) 

UndefVarError: UndefVarError: `V_set` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

# Model definition

## Initialize model instance

In [18]:
model= Model(HiGHS.Optimizer)

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

## Initialize variables

In [19]:
@variable(model, x[connection in A_set], Bin)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, [((5, 3, 1), (5, 3, 4)), ((4, 4, 1), (4, 4, 2)), ((0, 0, 0), (2, 3, 1)), ((2, 1, 1), (2, 1, 2)), ((0, 0, 0), (1, 2, 3)), ((1, 1, 2), (1, 1, 3)), ((0, 0, 0), (3, 1, 1)), ((3, 2, 2), (0, 0, 0)), ((5, 1, 5), (0, 0, 0)), ((3, 1, 2), (3, 1, 6))  …  ((1, 2, 4), (0, 0, 0)), ((1, 2, 1), (1, 2, 3)), ((5, 3, 2), (5, 3, 4)), ((1, 2, 3), (0, 0, 0)), ((0, 0, 0), (4, 2, 1)), ((1, 1, 4), (0, 0, 1)), ((1, 1, 3), (0, 0, 1)), ((2, 3, 1), (2, 3, 3)), ((0, 2, 0), (3, 1, 2)), ((3, 2, 1), (3, 2, 2))]
And data, a 51-element Vector{VariableRef}:
 x[((5, 3, 1), (5, 3, 4))]
 x[((4, 4, 1), (4, 4, 2))]
 x[((0, 0, 0), (2, 3, 1))]
 x[((2, 1, 1), (2, 1, 2))]
 x[((0, 0, 0), (1, 2, 3))]
 x[((1, 1, 2), (1, 1, 3))]
 x[((0, 0, 0), (3, 1, 1))]
 x[((3, 2, 2), (0, 0, 0))]
 x[((5, 1, 5), (0, 0, 0))]
 x[((3, 1, 2), (3, 1, 6))]
 ⋮
 x[((1, 2, 1), (1, 2, 3))]
 x[((5, 3, 2), (5, 3, 4))]
 x[((1, 2, 3), (0, 0, 0))]
 x[((0, 0, 0), (4, 2, 1))]
 x[((1, 1

## Objective function

In [20]:
@objective(model, Min, sum(x[connection] for connection in A_set if connection[1] == (0,0,0)))

x[((0, 0, 0), (2, 3, 1))] + x[((0, 0, 0), (1, 2, 3))] + x[((0, 0, 0), (3, 1, 1))] + x[((0, 0, 0), (5, 3, 1))] + x[((0, 0, 0), (5, 1, 3))] + x[((0, 0, 0), (2, 1, 1))] + x[((0, 0, 0), (1, 2, 1))] + x[((0, 0, 0), (4, 4, 1))] + x[((0, 0, 0), (3, 2, 1))] + x[((0, 0, 0), (3, 2, 4))] + x[((0, 0, 0), (1, 1, 1))] + x[((0, 0, 0), (4, 2, 1))]

## Initialize constraints

### Duplikate

In [21]:
@constraint(model, duplikat[connection in dup_start_set], x[(source_node, dup1_start)] + x[(source_node, dup2_start)] == 1)
@constraint(model, duplikat[connection in dup_end_set], x[(dup1_end, sink_node)] + x[(dup2_end, sink_node)] == 1)

UndefVarError: UndefVarError: `source_node` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Flusserhaltung

In [22]:
@constraint(model, 
    flow_conservation[node in V_flow],
    sum(x[connection] for connection in A_set if connection[2] == node) - 
    sum(x[connection] for connection in A_set if connection[1] == node) == 0
)


1-dimensional DenseAxisArray{Any,1,...} with index sets:
    Dimension 1, Tuple{Int64, Int64, Int64}[]
And data, a 0-element Vector{Any}

### All bus lines are served entirely

In [23]:
#@constraint(model,[connection in A_set; connection[1] != (0,0,0) && connection[1][1] == connection[2][1] && connection[1][2] == connection[2][2]],x[connection] == 1)

### Chat GPT

#### Übergeordnet

In [24]:
#Code wahrscheinlich redundant zu Zelle unterhalb
@constraint(model, sum(x[source_l', s] for s in [s1_l1, s1_l2]) == 1)
@constraint(model, sum(x[s, sink_l'] for s in [s2_l1, s2_l2]) == 1)

UndefVarError: UndefVarError: `s1_l1` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [25]:
@variable(model, x[edges], Bin)

# Quelle gibt 1 Einheit ab
@constraint(model, 
    sum(x[(:source_l, j)] for j in [:l1′, :l2′]) == 1)

# Senke nimmt 1 Einheit auf
@constraint(model, 
    sum(x[(j, :sink_l)] for j in [l1_end, l2_end]) == 1)


ErrorException: An object of name x is already attached to this model. If this
    is intended, consider using the anonymous construction syntax, for example,
    `x = @variable(model, [1:N], ...)` where the name of the object does
    not appear inside the macro.

    Alternatively, use `unregister(model, :x)` to first unregister
    the existing name from the model. Note that this will not delete the
    object; it will just remove the reference at `model[:x]`.


#### Fall B

In [26]:
#Schritt 3: Constraints für Fluss 1 von source → sink pro Paar

for (i, (dup1, dup2)) in enumerate(critical_pairs)
    src = Symbol("source_l", i)
    snk = Symbol("sink_l", i)

    end1 = Symbol("end_", dup1)
    end2 = Symbol("end_", dup2)

    # Nur ein Weg vom source zu einem der beiden Startknoten
    @constraint(model, x[(src, dup1)] + x[(src, dup2)] == 1)

    # Nur ein Weg von einem der beiden Enden zur sink
    @constraint(model, x[(end1, snk)] + x[(end2, snk)] == 1)
end


UndefVarError: UndefVarError: `critical_pairs` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

# Solve model

In [27]:
optimize!(model)

Running HiGHS 1.8.0 (git hash: fcfb53414): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+00]
Presolving model
0 rows, 0 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               0                  0.00%        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      0
  Dual bound        0
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    0 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.00 (total)
                    0.00 (presolve)
        

In [28]:
println("Termination status: ", termination_status(model))
println("Primal status: ", primal_status(model))

if status == MOI.OPTIMAL
    println("Optimale Lösung gefunden!")
    println("Zielfunktionswert: ", objective_value(model))
    active_connections = []
    for connection in A_set 
        if value(x[connection]) > 1e-6
            push!(active_connections, connection)
        end
    end
    sort(active_connections, by = x -> x[1][1][1])
    full_output = join(string.(active_connections), "\n")
    println(full_output)

else
    println("Kein Optimum – Solver-Status: ", status)
end
    

Termination status: OPTIMAL
Primal status: FEASIBLE_POINT


UndefVarError: UndefVarError: `status` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name also exists in Pkg.
Hint: a global variable of this name also exists in DataStructures.

# Visualization

In [29]:
# Store full node struct as value for lookup
nodes_dict = Dict{Tuple{Int, Int, Int}, typeof(nodes[1])}()
for connection in active_connections
    origin,goal=connection
    a,b,c = origin
    d,e,f = goal
    node=findfirst(x -> x.bus_line_id == a && x.line_id == b && x.stop_id == c, nodes)
    if !(node in keys(nodes_dict))
        nodes_dict[origin] = nodes[node]
    end
    node=findfirst(x -> x.bus_line_id == d && x.line_id == e && x.stop_id == f, nodes)
    if !(node in keys(nodes_dict))
        nodes_dict[goal] = nodes[node]
    end
end
sort(collect(nodes_dict))

UndefVarError: UndefVarError: `active_connections` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Plot

In [30]:
# 1. Prepare node coordinates and hover labels
xs = [n.coord_x for n in nodes]
ys = [n.coord_y for n in nodes]
zs = [n.start_time for n in nodes]

labels = [string("(", n.bus_line_id, ", ", n.line_id, ", ", n.stop_id, ")\n", "t=", round(n.start_time, digits=1)) for n in nodes]

trace_nodes = scatter3d(
    x=xs, y=ys, z=zs,
    mode="markers",
    marker=attr(size=3, color="blue"),
    text=labels,
    hoverinfo="text",
    name="Nodes"
)

# 2. Extract active connections from the model
active_connections = [(i, j) for (i, j) in A_set if value(x[(i, j)]) > 1e-6]

# 3. Generate path-based edges
traces_edges = []

for (start_key, end_key) in active_connections
    start_bus, start_line, _ = start_key
    end_bus, end_line, _ = end_key

    if start_bus == end_bus && start_line == end_line
        # Get all stops for this line
        path_nodes = sort(
            filter(n -> n.bus_line_id == start_bus && n.line_id == start_line, nodes),
            by = n -> n.stop_id
        )

        push!(traces_edges, scatter3d(
            x = [n.coord_x for n in path_nodes],
            y = [n.coord_y for n in path_nodes],
            z = [n.start_time for n in path_nodes],
            mode = "lines",
            line = attr(width=2),
            hoverinfo = "none",
            showlegend = false
        ))
    else
        # Just draw a direct line between tours (e.g., depot → first stop or end → new line)
        x_vals = [nodes_dict[start_key].coord_x, nodes_dict[end_key].coord_x]
        y_vals = [nodes_dict[start_key].coord_y, nodes_dict[end_key].coord_y]
        z_vals = [nodes_dict[start_key].start_time, nodes_dict[end_key].start_time]

        push!(traces_edges, scatter3d(
            x = x_vals, y = y_vals, z = z_vals,
            mode = "lines",
            line = attr(width=2, dash="dot"),
            hoverinfo = "none",
            showlegend = false
        ))
    end
end

# 4. Combine and plot
layout = Layout(
    title="Tour Sequences with Full Paths",
    scene=attr(
        xaxis=attr(title="X"),
        yaxis=attr(title="Y"),
        zaxis=attr(title="Start Time")
    ),
    showlegend=false
)

plot_obj = Plot([trace_nodes; traces_edges...], layout)

# 5. Save plot to interactive HTML file
PlotlyJS.savefig(plot_obj,"/Users/alexanderklaus/Desktop/Masterthesis/Code/output/3d_plot_new.html")


"/Users/alexanderklaus/Desktop/Masterthesis/Code/output/3d_plot_new.html"